In [5]:
import os
import requests
import json

url = "http://oqmd.org/oqmdapi/formationenergy?filter=element_set=(Al-Fe)&format=json"

response = requests.get(url)

# Ensure we fail fast if there was an error with the request.
response.raise_for_status()

data = response.json()

# Check if the file exists
filename = 'outfile.json'
if os.path.exists(filename):
    # If it exists, load the current data
    with open(filename, 'r') as f:
        current_data = json.load(f)
    # Combine current data with new data
    combined_data = {**current_data, **data}
else:
    combined_data = data

# Write the combined data to the file
with open(filename, 'w') as f:
    json.dump(combined_data, f, indent=4) # indent parameter added here


## Data Extraction using API Request

In [6]:
import pandas as pd
import requests

url = "http://oqmd.org/oqmdapi/formationenergy?filter=element_set%3D%28Al-Fe%29&format=json"

response = requests.get(url)


response.raise_for_status()

data = response.json()


df = pd.json_normalize(data['data'])


for column in df.columns:
    if isinstance(df[column][0], list):
        
        df[column] = df[column].apply(lambda x: ', '.join(map(str, x)))
    elif isinstance(df[column][0], dict):
        df = df.join(pd.json_normalize(df[column]).add_prefix(f'{column}_'))
        df = df.drop(columns=column)
df.to_csv('data.csv', index=False)


## Allowing only Stable Composition

In [8]:
import requests
import pandas as pd

url = "http://oqmd.org/oqmdapi/formationenergy?filter=element_set%3D%28Al-Fe%29&format=json&limit=50&offset=50"
response = requests.get(url)

data = response.json()["data"]

stable_compositions = [entry for entry in data if entry["stability"] <= 0]

df = pd.DataFrame(stable_compositions)

df.to_csv('stable_compositions.csv', index=False)


## Adding Chemical Composition to the chemicla file which I've genearted 


In [11]:
import pandas as pd
import numpy as np

# Load the existing CSV file
df = pd.read_csv('intern_data.csv')

# Prepare the symbols list
symbols = ['Os', 'Ir', 'Pt', 'Au', 'Ag', 'Cu', 'Al', 'Sn', 'Pb', 'Hg', 'Ga', 'In', 
           'Tl', 'Bi', 'Po', 'At', 'Rn', 'Fr', 'Ra', 'Ac', 'Th', 'Pa', 'U', 'Np', 
           'Pu', 'Am', 'Cm', 'Bk', 'Cf', 'C', 'V2O5', 'TiN', 'BN', 'W', 'Hf', 'Ta', 
           'C', 'C', 'Ni', 'Si', 'F2C']

# Extend the list with None to match the number of rows in the dataframe
symbols.extend([None] * (len(df) - len(symbols)))

# Add the new column
df['Chemical_Symbol'] = symbols

# Save the updated DataFrame back to a new CSV file
df.to_csv('updated_intern_data.csv', index=False)


In [13]:
import pandas as pd

# Read your data into a pandas DataFrame
df = pd.read_csv('updated_intern_data.csv')

# Define a dictionary with the material names and corresponding chemical symbols
missing_symbols = {
    'Magnesium': 'Mg',
    'Titanium': 'Ti',
    'Carbon fiber': 'C',
    'Kevlar': 'C14H10N2O2',
    'Aramid': 'C12H10N2O2',
    'Glass fiber': 'SiO2',
    'Epoxy': 'C21H25ClO5',
    'Polyurethane': '(C27H36N2O10)n',
    'Vinyl ester': 'C9H10O3',
    'Polyester': '(C10H8O4)n',
    'Peek': 'C19H24O3',
    'Nylon': '(C6H11NO)n',
}

# Update the 'Chemical_Symbol' column for each material in the dictionary
for material, symbol in missing_symbols.items():
    df.loc[df['Material'] == material, 'Chemical_Symbol'] = symbol

# Save the updated DataFrame back to the CSV file
df.to_csv('updated_intern_data.csv', index=False)


In [14]:
import pandas as pd

# Load the CSV data
data = pd.read_csv('updated_intern_data.csv')

# Define base url
base_url = 'http://oqmd.org/oqmdapi/formationenergy'

# Iterate over the data and create URLs
urls = []
for index, row in data.iterrows():
    symbol = row['Chemical_Symbol']
    url = f"{base_url}?fields=name,entry_id,delta_e&filter=element={symbol}&format=json"
    urls.append(url)

# Print the generated urls
for url in urls:
    print(url)


http://oqmd.org/oqmdapi/formationenergy?fields=name,entry_id,delta_e&filter=element=Os&format=json
http://oqmd.org/oqmdapi/formationenergy?fields=name,entry_id,delta_e&filter=element=Ir&format=json
http://oqmd.org/oqmdapi/formationenergy?fields=name,entry_id,delta_e&filter=element=Pt&format=json
http://oqmd.org/oqmdapi/formationenergy?fields=name,entry_id,delta_e&filter=element=Au&format=json
http://oqmd.org/oqmdapi/formationenergy?fields=name,entry_id,delta_e&filter=element=Ag&format=json
http://oqmd.org/oqmdapi/formationenergy?fields=name,entry_id,delta_e&filter=element=Cu&format=json
http://oqmd.org/oqmdapi/formationenergy?fields=name,entry_id,delta_e&filter=element=Al&format=json
http://oqmd.org/oqmdapi/formationenergy?fields=name,entry_id,delta_e&filter=element=Sn&format=json
http://oqmd.org/oqmdapi/formationenergy?fields=name,entry_id,delta_e&filter=element=Pb&format=json
http://oqmd.org/oqmdapi/formationenergy?fields=name,entry_id,delta_e&filter=element=Hg&format=json
http://oqm

In [15]:
import requests

# Initialize an empty dataframe to store the responses
df_responses = pd.DataFrame()

# Iterate over the URLs and make requests
for url in urls:
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()

        # Check if the data is not empty
        if 'data' in data:
            df_temp = pd.DataFrame(data['data'])
            
            # Append the response data to our dataframe
            df_responses = pd.concat([df_responses, df_temp])

# Save the responses to a new CSV file
df_responses.to_csv('output_data.csv', index=False)


## Building API URL Request Base don the CSv file Chemical Composioton

In [1]:
import pandas as pd
import requests

# Load the CSV data
data = pd.read_csv('updated_intern_data.csv')

# Define base url
base_url = 'http://oqmd.org/oqmdapi/formationenergy'

# Iterate over the data and create URLs
urls = []
for index, row in data.iterrows():
    symbol = row['Chemical_Symbol']
    url = f"{base_url}?fields=name,entry_id,delta_e&filter=element={symbol}&format=json"
    urls.append(url)

# Print the generated urls
# for url in urls:
#     print(url)


# Initialize an empty dataframe to store the responses
df_responses = pd.DataFrame()

# Iterate over the URLs and make requests
for url in urls:
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()

        # Check if the data is not empty and 'data' key is in the JSON response
        if 'data' in data and data['data']:
            # Pandas version 1.0.0 and later
            if pd.__version__ >= '1.0.0':
                df_temp = pd.json_normalize(data['data'])
            # Pandas versions earlier than 1.0.0
            else:
                df_temp = pd.io.json.json_normalize(data['data'])

            # Append the response data to our dataframe
            df_responses = pd.concat([df_responses, df_temp])

# Save the responses to a new CSV file
df_responses.to_csv('output_data.csv', index=False)


http://oqmd.org/oqmdapi/formationenergy?fields=name,entry_id,delta_e&filter=element=Os&format=json
http://oqmd.org/oqmdapi/formationenergy?fields=name,entry_id,delta_e&filter=element=Ir&format=json
http://oqmd.org/oqmdapi/formationenergy?fields=name,entry_id,delta_e&filter=element=Pt&format=json
http://oqmd.org/oqmdapi/formationenergy?fields=name,entry_id,delta_e&filter=element=Au&format=json
http://oqmd.org/oqmdapi/formationenergy?fields=name,entry_id,delta_e&filter=element=Ag&format=json
http://oqmd.org/oqmdapi/formationenergy?fields=name,entry_id,delta_e&filter=element=Cu&format=json
http://oqmd.org/oqmdapi/formationenergy?fields=name,entry_id,delta_e&filter=element=Al&format=json
http://oqmd.org/oqmdapi/formationenergy?fields=name,entry_id,delta_e&filter=element=Sn&format=json
http://oqmd.org/oqmdapi/formationenergy?fields=name,entry_id,delta_e&filter=element=Pb&format=json
http://oqmd.org/oqmdapi/formationenergy?fields=name,entry_id,delta_e&filter=element=Hg&format=json
http://oqm

In [2]:
import requests
import json

# Specify the URL
url = "http://oqmd.org/oqmdapi/formationenergy"

# Specify the parameters
params = {
    "fields": "name,entry_id,spacegroup,ntypes,band_gap,delta_e",
    "filter": "element_set=(Al-Fe),O",
    "limit": 2,
    "offset": 2
}

# Send the request and capture the response
response = requests.get(url, params=params)

# Check the status code to verify that the request was successful
if response.status_code == 200:
    print("Request was successful")
    data = response.json()
    with open('output.json', 'w') as f:
        json.dump(data, f, indent=4)  # Use indent for pretty-print
else:
    print("Request failed with status code", response.status_code)


Request was successful


## Extraction of Chemial Based on the Lsit

In [2]:
import requests
import pandas as pd
from pandas import json_normalize


# Elements list
elements = ["Os", "Ir", "Pt", "Au", "Ag", "Cu", "Al", "Sn", "Pb", "Hg", "Ga", "In", "Tl", 
            "Bi", "Po", "At", "Rn", "Fr", "Ra", "Ac", "Th", "Pa", "U", "Np", "Pu", "C", "C9H10O3", 
            "(C10H8O4)n", "C19H24O3", "(C6H11NO)n"]

# Base URL
base_url = "http://oqmd.org/oqmdapi/formationenergy"

# Initialize an empty DataFrame to store the results
df_final = pd.DataFrame()

# For each element
for element in elements:
    # Specify the parameters
    params = {
        "fields": "name,entry_id,spacegroup,ntypes,band_gap,delta_e",
        "filter": f"element_set=({element})",
        "limit": 2,
        "offset": 2
    }
    
    # Send a GET request to the URL with specified parameters
    response = requests.get(base_url, params=params)
    
    # Convert the response from JSON to a Python dictionary
    data = response.json()
    
    # Check if the 'data' key is in the JSON object
    if 'data' in data.keys():
        # Normalize semi-structured JSON data into a flat table.
        df = json_normalize(data, 'data')
        
        # Append the DataFrame to the final DataFrame
        df_final = df_final.append(df, ignore_index=True)

# Save the DataFrame as a CSV file
df_final.to_csv('output.csv', index=False)


In [4]:
import os
import requests
import json

# Elements list
elements = ["Os", "Ir", "Pt", "Au", "Ag", "Cu", "Al", "Sn", "Pb", "Hg", "Ga", "In", "Tl", 
            "Bi", "Po", "At", "Rn", "Fr", "Ra", "Ac", "Th", "Pa", "U", "Np", "Pu", "C", "C9H10O3", 
            "(C10H8O4)n", "C19H24O3", "(C6H11NO)n"]

# Base URL
base_url = "http://oqmd.org/oqmdapi/formationenergy"

# Check if the file exists
filename = 'outfile.json'
if os.path.exists(filename):
    # If it exists, load the current data
    with open(filename, 'r') as f:
        current_data = json.load(f)
else:
    current_data = {}

# For each element
for element in elements:
    # Generate the URL
    url = f"{base_url}?filter=element_set=({element})&format=json"
    
    try:
        # Send a GET request to the URL
        response = requests.get(url)

        # Ensure we fail fast if there was an error with the request.
        response.raise_for_status()
        
        # Convert the response from JSON to a Python dictionary
        data = response.json()
        
        # Combine current data with new data
        combined_data = {**current_data, **data}
    except requests.exceptions.HTTPError as err:
        print(f"HTTP error occurred for element {element}: {err}")
        continue

# Write the combined data to the file
with open(filename, 'w') as f:
    json.dump(combined_data, f, indent=4) # indent parameter added here


HTTP error occurred for element C9H10O3: 501 Server Error: Not Implemented for url: https://oqmd.org:443/oqmdapi/formationenergy?filter=element_set=(C9H10O3)&format=json
HTTP error occurred for element (C10H8O4)n: 400 Client Error: Bad Request for url: https://oqmd.org:443/oqmdapi/formationenergy?filter=element_set=((C10H8O4)n)&format=json
HTTP error occurred for element C19H24O3: 501 Server Error: Not Implemented for url: https://oqmd.org:443/oqmdapi/formationenergy?filter=element_set=(C19H24O3)&format=json
HTTP error occurred for element (C6H11NO)n: 400 Client Error: Bad Request for url: https://oqmd.org:443/oqmdapi/formationenergy?filter=element_set=((C6H11NO)n)&format=json


In [6]:
import pandas as pd
import json

# Load the data from the JSON file
with open('outfile.json', 'r') as f:
    data = json.load(f)

# Extract the data elements
data_elements = data['data']

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(data_elements)

# Filter for only stable elements. Here we're assuming stable means "stability" is less than 0.5.
stable_df = df[df['stability'] < 0.5]

# Write the DataFrame to a CSV file
stable_df.to_csv('output_stable.csv', index=False)


In [14]:
import pandas as pd
import requests
import os
import json

# Load the data from your CSV file
df = pd.read_csv('updated_intern_data.csv', delimiter=',')

# Get unique materials/chemical symbols
unique_materials = df['Chemical_Symbol'].unique().tolist()

# Base URL
base_url = "http://oqmd.org/oqmdapi/formationenergy"

# Check if the file exists
filename = 'outfile.json'
if os.path.exists(filename):
    # If it exists, load the current data
    with open(filename, 'r') as f:
        current_data = json.load(f)
else:
    current_data = {}

# Loop through each unique material
for material in unique_materials:
    try:
        # Generate the URL
        url = f"{base_url}?filter=element_set=({material})&format=json"
        
        # Make the GET request
        response = requests.get(url)
        
        # Check if the request was successful
        if response.status_code == 200:
            # Save the result
            current_data[material] = response.json()
        else:
            print(f"Request for {material} failed with status code: {response.status_code}")

    except Exception as e:
        print(f"Error occurred while fetching data for {material}: {str(e)}")

# Save the updated data to the file
with open(filename, 'w') as f:
    json.dump(current_data, f)

# Now 'current_data' is a dictionary with chemical symbols as keys and fetched data as values


Request for V2O5 failed with status code: 501
Request for TiN failed with status code: 501
Request for BN failed with status code: 501
Request for F2C failed with status code: 501
Request for nan failed with status code: 400
Request for C14H10N2O2 failed with status code: 501
Request for C12H10N2O2 failed with status code: 501
Request for SiO2 failed with status code: 501
Request for C21H25ClO5 failed with status code: 501
Request for (C27H36N2O10)n failed with status code: 400
Request for C9H10O3 failed with status code: 501
Request for (C10H8O4)n failed with status code: 400
Request for C19H24O3 failed with status code: 501
Request for (C6H11NO)n failed with status code: 400


## proper Format

In [16]:
import json

# Load the JSON data
with open('outfile.json', 'r') as f:
    data = json.load(f)

# Reformat the JSON data with proper indentation and sorting of keys
formatted_data = json.dumps(data, indent=4, sort_keys=True)

# Write the formatted JSON data back to the file
with open('outfile.json', 'w') as f:
    f.write(formatted_data)


In [25]:
import pandas as pd
import json

# Load the JSON data
with open('outfile.json', 'r') as f:
    data = json.load(f)

flattened_data = []
for key, value in data.items():
    if isinstance(value, dict):
        for sub_key, sub_value in value.items():
            if isinstance(sub_value, list):
                for entry in sub_value:
                    # Add the parent key to the entry dictionary
                    entry[key] = sub_key
                    flattened_data.append(entry)

# Convert the list of dictionaries into a DataFrame
df = pd.DataFrame(flattened_data)

# Write the DataFrame to a CSV file
df.to_csv('outfile.csv', index=False)

In [29]:
import csv
import json

# Load JSON data from the file
with open('outfile.json', 'r') as f:
    json_data = json.load(f)

filename = "your_file.csv"

# Open the CSV file for writing
with open(filename, 'w', newline='') as csvfile:
    
    # As we don't know the fields beforehand, we take an example from the first element
    first_element = json_data['data'][0]
    # Inserting 'name' at the start of the fieldnames
    fieldnames = ['name'] + list(first_element.keys())
    
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()  # write the header

    # Loop over JSON data
    for entry in json_data['data']:
        row = {'name': 'Ac'}  # create the first field, assuming 'Ac' as the name
        row.update(entry)  # add the rest of the fields
        writer.writerow(row)  # write the row to the CSV file


In [31]:
import requests
import csv
import time

# Function to fetch data from the API using chemical symbol
def fetch_data(api_url, chemical_symbol):
    params = {'chemical_symbol': chemical_symbol}
    try:
        response = requests.get(api_url, params=params)
        response.raise_for_status()
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Failed to fetch data for {chemical_symbol}: {response.status_code}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Request for {chemical_symbol} failed with error: {e}")
        return None

# API URL to fetch data
api_url = "http://oqmd.org/oqmdapi/formationenergy"

# Read the chemical symbols from the CSV file
chemical_symbols = []
with open('updated_intern_data.csv', 'r') as csv_file:
    reader = csv.DictReader(csv_file)
    for row in reader:
        chemical_symbol = row['Chemical_Symbol'].strip()
        if chemical_symbol:
            chemical_symbols.append(chemical_symbol)

# Fetch data for each chemical symbol and store in a list
data_list = []
for chemical_symbol in chemical_symbols:
    data = fetch_data(api_url, chemical_symbol)
    if data:
        data_list.append(data)
    time.sleep(0.5)  # Add a delay of 0.5 seconds between API requests

# Extracting the field names from the first entry in the data list
field_names = list(data_list[0].keys())

# Writing the data into a new CSV file
output_file = 'material_data_from_api.csv'
with open(output_file, 'w', newline='') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=field_names)
    writer.writeheader()
    for data in data_list:
        writer.writerow(data)

print("Data has been fetched and written to", output_file)


Data has been fetched and written to material_data_from_api.csv


In [33]:
df = pd.DataFrame(data_list)

# Writing the DataFrame to a new CSV file
output_file = 'material_data_from_api.csv'
df.to_csv(output_file, index=False)

In [37]:
formatted_json = json.dumps(data_list, indent=4)

# Save the formatted JSON data to a file
with open('data.json', 'w') as json_file:
    json_file.write(formatted_json)